# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display, HTML


from core.downloads.geosampa import get_capabilities, get_features

In [ ]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [ ]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [ ]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

In [ ]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip